# Create a 2D Surface Grain Boundary in a film

Use commensurate lattice matching algorithm to create grain boundaries in 2D materials by finding appropriate twist angles between two phases of the same material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete (depending on the parameters can take a few min).
1. Scroll down to view results.

## Summary

1. Prepare the Environment: Set up the notebook and install packages, preview the input material
1. Create and visualize the initial material
1. Generate grain boundaries with commensurate lattice matching
1. Select the grain boundary with the desired twist angle and visualize it

## Notes

1. We perform commensurate lattice matching to find valid supercells that achieve the desired twist angle.
1. When the matching is finished, grain boundaries with angles close to the target are presented.
1. The algorithm searches for supercell matrices within specified size limits.
1. Edge atoms are handled specially with inclusion and distance tolerances.
1. For more information, see the [Introduction](Introduction.ipynb) notebook.


## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior


In [17]:
# Material selection
MATERIAL_INDEX = 1  # Index in the list of materials

# Grain boundary parameters
TARGET_TWIST_ANGLE = 17.9  # in degrees
INTERFACE_GAP = 2.0  # in Angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]

# Search algorithm parameters
MAX_REPETITION = 6  # Maximum supercell matrix element value
ANGLE_TOLERANCE = 2.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# How much of the edge region to keep for left material to overlap in the interfacial region
EDGE_INCLUSION_TOLERANCE = 1.0  # in Angstroms

# Distance tolerance for atoms to be considering too close when merging, and previous one be removed.  
# Should be less than the expected bond length
DISTANCE_TOLERANCE = 1.2  # in Angstroms

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [3, 3, 1]

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [18]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("", "../../config.yml")

### 1.3. Load and preview input material

In [19]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

0: Data from 0-Ni has been read successfully.
1: Data from 1-Graphene has been read successfully.
2: Data from 10-Al2O3, Sapphire, RHL (R-3c) 3D (Bulk), mp-1143 (1) has been read successfully.
3: Data from 11-ZnO, Zinc Oxide, HEX (P6_3mc) 3D (Bulk), mp-2133 has been read successfully.
4: Data from 12-Cd4 Te4 has been read successfully.
5: Data from 13-Si4 C4 has been read successfully.
6: Data from 14-GaN, Gallium Nitride, HEX (P6_3mc) 3D (Bulk), mp-804 has been read successfully.
7: Data from 15-WS2, Tungsten Disulfide, HEX (P-6m2) 2D (Monolayer), 2dm-3749 has been read successfully.
8: Data from 16 - In4 P4 has been read successfully.
9: Data from 17 - Silicon FCC (8) has been read successfully.
10: Data from 4-Te2Mo has been read successfully.
11: Data from 5-HfO2 has been read successfully.
12: Data from 6-Ni4(110), termination Ni_Pmmm_2, Slab, Terrace, 1 steps, [2 0 0] has been read successfully.
13: Data from 7-Ag4 has been read successfully.
14: Data from 8-Si, Silicene, HEX (P-

## 2. Prepare Material
### 2.1. Select and visualize initial material

In [20]:
from utils.visualize import visualize_materials

material = materials[MATERIAL_INDEX]

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=VISUALIZE_REPETITIONS)

GridBox(children=(VBox(children=(Label(value='C2 - Material - rotation: 0x,0y,0z', layout=Layout(align_self='c…

## 3. Generate Surface Grain Boundary
### 3.1. Set up grain boundary configuration and builder


In [21]:
from mat3ra.made.tools.build.grain_boundary import (
    SurfaceGrainBoundaryConfiguration,
    SurfaceGrainBoundaryBuilderParameters,
    SurfaceGrainBoundaryBuilder
)

config = SurfaceGrainBoundaryConfiguration(
    film=material,
    twist_angle=TARGET_TWIST_ANGLE,
    distance_z=INTERFACE_GAP,
    gap=INTERFACE_GAP,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX
)

params = SurfaceGrainBoundaryBuilderParameters(
    max_supercell_matrix_int=MAX_REPETITION,
    angle_tolerance=ANGLE_TOLERANCE,
    return_first_match=RETURN_FIRST_MATCH,
    edge_inclusion_tolerance=EDGE_INCLUSION_TOLERANCE,
    distance_tolerance=DISTANCE_TOLERANCE
)

builder = SurfaceGrainBoundaryBuilder(build_parameters=params)

### 3.2. Generate and analyze grain boundaries


In [22]:
from utils.plot import plot_twisted_interface_solutions

grain_boundaries = builder.get_materials(config)

print(f"\nFound {len(grain_boundaries)} possible grain boundary structures")
for i, gb in enumerate(grain_boundaries):
    actual_angle = gb.metadata.get("actual_twist_angle", "unknown")
    print(f"\nGrain Boundary {i + 1}:")
    print(f"Actual twist angle: {actual_angle}°")
    print(f"Number of atoms: {len(gb.basis.elements.ids)}")

if len(grain_boundaries) > 0:
    plot_twisted_interface_solutions(grain_boundaries)

Trying max_int = 6
Found commensurate lattice with angle 17.897 and size metric 31.0

Found 1 possible grain boundary structures

Grain Boundary 1:
Actual twist angle: unknown°
Number of atoms: 126


## 4. Preview the selected grain boundary
By default, the first grain boundary is selected. You can change the selection by changing the `selected_gb` index.

In [23]:
selected_gb = grain_boundaries[0]
actual_angle = selected_gb.metadata.get("build").get("configuration").get("actual_twist_angle")
print(f"Target angle: {TARGET_TWIST_ANGLE}°")
print(f"Actual angle: {actual_angle}°")
print(f"Number of atoms: {len(selected_gb.basis.elements.ids)}")

visualize_materials(selected_gb, repetitions=[1, 1, 1])
visualize_materials(selected_gb, repetitions=[1, 1, 1], rotation="-90x")

Target angle: 17.9°
Actual angle: None°
Number of atoms: 126


GridBox(children=(VBox(children=(Label(value='C126 - Material - rotation: 0x,0y,0z', layout=Layout(align_self=…

GridBox(children=(VBox(children=(Label(value='C126 - Material - rotation: -90x', layout=Layout(align_self='cen…

### 5. Pass data to the outside runtime


In [24]:
from utils.jupyterlite import set_materials

set_materials(selected_gb)

Data for materials written to uploads/C, Graphene, HEX (P6:mmm) 2D (Monolayer), 2dm-3993, Grain Boundary (17.90°).json
